In [23]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import copy
import scipy.stats
from scipy import signal
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as xccrs
import cartopy.feature as cfeature
%matplotlib inline

from utils import calendar_utils as cal

import momlevel

import matplotlib.ticker as mticker
panellabs=list(map(chr, range(97, 123)))

# %matplotlib inline

# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "Helvetica"
# })

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data I/O using Dask: probably necessary if more than ~30 tgs

In [22]:
cluster.close()
client.close()
client.shutdown()

In [5]:
def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=1,
        memory='20GB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=20GB',
        project='UARH0002',
        walltime='02:00:00',
        interface='ib0',)

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status',
        'array.slicing.split_large_chunks': True
    })
    client = Client(cluster)
    return cluster, client

cluster, client = get_ClusterClient()
cluster.scale(10) 

/glade/work/clittle/conda-envs/sealevel-newmomlevel/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile
/glade/work/clittle/conda-envs/sealevel-newmomlevel/lib/python3.8/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43618 instead
  warnings.warn(


In [6]:
cluster

## read in monthly SSH files for an HR cycle

In [7]:
import os, re

def get_filenames_in_range(path_to_directory, lower_bound, upper_bound):
    files = []
    # Iterate through files in current directory
    for f in os.listdir(path_to_directory):
        if f.endswith(".nc"):            # print(f)        
            # Perform a regex match for numbers
            match = re.findall(r'\d+', f)
            # If numbers are found, check whether they are within range
            if match:
                number = int(match[-2])
                if number >= lower_bound and number <= upper_bound:
                    files.append(os.path.join(diri, f))
    return files

def preprocessor(d0):
    d0 = d0[['SSH']]
    d0 = cal.time_set_midmonth(d0,'time')
    # d0 = d0.groupby('time.year').mean('time').rename({'year':'time'})
    return d0

#HR cycle 3 #1993-2018
ncycle=3
startyear=1993-1958+1
nyears=2018-1993
startnumber=startyear+(ncycle-1)*61
diri = '/glade/campaign/collections/cmip/CMIP6/iHESP/FOSI/HR/g.e20.G.TL319_t13.control.001_contd/ocn/hist'
fin=get_filenames_in_range(diri,startnumber,startnumber+nyears)
len(fin)/12

26.0

In [8]:
ds = xr.open_mfdataset(fin,combine='by_coords',coords="minimal",compat="override",
                       preprocess=preprocessor,
                       parallel=True,)

In [9]:
ds

<xarray.Dataset>
Dimensions:  (time: 312, nlat: 2400, nlon: 3600)
Coordinates:
  * time     (time) object 0158-01-15 00:00:00 ... 0183-12-15 00:00:00
    ULONG    (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    ULAT     (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    TLONG    (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
    TLAT     (nlat, nlon) float64 dask.array<chunksize=(2400, 3600), meta=np.ndarray>
Dimensions without coordinates: nlat, nlon
Data variables:
    SSH      (time, nlat, nlon) float32 dask.array<chunksize=(1, 2400, 3600), meta=np.ndarray>
Attributes:
    title:             g.e20.G.TL319_t13.control.001_contd
    history:           none
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    time_period_freq:  day_5
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    contents:          Diagnostic and Prognostic Variables
    source:            CCSM POP2, the CCSM Ocean Component
    revision:          $Id: tavg.F90 90507 2019-01-18 20:54:19Z altuntas@ucar...
    calendar:          All years have exactly  365 days.
    start_time:        This dataset was created on 2019-09-20 at 15:51:47.9
    cell_methods:      cell_methods = time: mean ==> the variable values are ...

In [10]:
fosi=ds.where(ds.SSH < 1e5)
geolon=fosi.TLONG.drop_vars(['TLONG','TLAT','ULONG','ULAT'])
geolat=fosi.TLAT.drop_vars(['TLONG','TLAT','ULONG','ULAT'])
omask=xr.where(np.isnan(fosi.SSH[0,:,:]), 0, 1)
omask=omask.drop_vars(['TLONG','TLAT','ULONG','ULAT'])

### process FOSI at TG locations

In [24]:
tgcsvin="./real_tgs_in_small.csv"
# tgcsvin="./pseudo_tgs_in.csv"
tgs_in=pd.read_csv(tgcsvin)
tgs_in

,name,lat,lon,name2,coast
0,10,37.806667,-122.465000,SAN FRANCISCO,823
1,234,32.781667,-79.925000,CHARLESTON I,960
2,12,40.700000,-74.013333,NEW YORK (THE BATTERY),960
3,235,42.353333,-71.053333,BOSTON,960


In [25]:
fosi_tg_ts=momlevel.extract_tidegauge(fosi.SSH.drop_vars(['TLONG','TLAT','ULONG','ULAT']), geolon, geolat, mask=omask, csv=tgcsvin)

In [26]:
fosi_in=tgs_in.copy()
fosi_in.index=fosi_in['name']
fosi_in=fosi_in.T

for colname in fosi_in:
    fosi_in[colname].lat=fosi_tg_ts[colname].model_coords[0]
    fosi_in[colname].lon=fosi_tg_ts[colname].model_coords[1]
    # print(colname)
fosi_in.to_pickle("./real_hr3_tg_locs_small.pkl")
# fosi_in.to_pickle("./pseudo_hr3_tg_locs.pkl")

In [ ]:
%%time

tg_ibcorr=pd.read_pickle("tg_ts_93_18_small.pkl")

fosi_ibcorr=fosi_tg_ts.to_dataframe()
fosi_ibcorr=fosi_ibcorr.T
fosi_ibcorr.index=fosi_ibcorr.index.astype('str')
fosi_ibcorr=fosi_ibcorr.T
fosi_ibcorr.columns
fosi_ibcorr.index=tg_ibcorr.index

fosi_ibcorr.to_pickle("./real_hr3_ts_small.pkl")
# fosi_ibcorr.to_pickle("./pseudo_hr3_ts.pkl")